In [10]:
from utils.util import *
from ultralytics import RTDETR
from PIL import Image, ImageDraw, ImageFont
from pathlib import Path

In [11]:
detector = RTDETR("../models/object_detection.pt")

classifier = get_pretrained_resnet(num_classes=1, pretrained=False)
classifier.load_state_dict(torch.load("../models/defect_detection_model.pth"))
classifier.to(DEVICE)

detectable_classes = {7:0, 5:1, 11:2, 1:4}
class_to_model = {0 : "defect_detection_glass_model.pth", 1 : "defect_detection_lightning_model.pth", 2: "defect_detection_polymer_model.pth", 4: "defect_detection_yoke_model.pth"}
class_to_problem = {0 : "Missing cap", 1 : "Rust", 2: "Rust", 4: "Rust"}

ResNet18 loaded. Final layer replaced for 1 output features.
Only the final layer will be trained initially.


In [12]:
def get_model(class_id):
    model_path = class_to_model[class_id]
    model_path = os.path.join("../models", model_path)
    model = get_pretrained_resnet(num_classes=1, pretrained=False)
    model.load_state_dict(torch.load(model_path))
    model.to(DEVICE)
    model.eval()

    return model

In [35]:
def run_full_pipeline(img, image_path:str|Path, save_dir:Path|None=None, pad:int=0):
    """
    • Detect parts with REDETR
    • Crop each part
    • Run defect classifier on the crops
    • Optionally save crops for inspection
    """
    original_img = img

    if img is None:
        original_img = Image.open(image_path).convert("RGB")

    img_defections   = original_img.copy()          # red boxes will go here
    draw    = ImageDraw.Draw(img_defections)

    results = detector.predict(original_img)
    detections = []
    for result in results:
        print(f"  Type of result.boxes: {type(result.boxes)}")
        print(f"  Value of result.boxes: {result.boxes}")
        if result.boxes is not None and len(result.boxes) > 0:
            boxes = result.boxes
            confidences = boxes.conf
            cls_indices = boxes.cls

            print(f"  Type of boxes: {type(boxes)}")
            print(f"  Type of confidences: {type(confidences)}")
            print(f"  Type of cls_indices: {type(cls_indices)}")

            for box, confidence, class_id in zip(boxes.xyxy, confidences, cls_indices):
                x1, y1, x2, y2 = box.tolist()
                confidence_value = confidence.item()
                class_name = result.names[int(class_id)]
                print(f"    Detected {class_name} with confidence: {confidence_value:.2f} at ({x1:.0f}, {y1:.0f}), ({x2:.0f}, {y2:.0f})")
                if class_id in detectable_classes:
                    id = detectable_classes[class_id]
                    crop = crop_object(original_img, box)
                    model = get_model(id)
                    prob, label = predict_single(crop, model, DEVICE)
                    if label != 0:
                        print(f"    Detected {class_to_problem[id]} with confidence: {prob:.2f}")
                        draw.rectangle([(x1, y1), (x2, y2)], outline="red", width=1)
                        draw.text((x1, y1 - 12), class_to_problem[id], fill="red")
                        detections.append({"id": id, "prob": prob, "label": label, "type": class_to_problem[id], "box": box, "confidence": confidence})


            img_object_detection = Image.fromarray(result.plot().astype('uint8')) # Convert to PIL Image
            save_path = f"./{save_dir}/object_{os.path.basename(image_path)}"
            save_path_d = f"./{save_dir}/detected_{os.path.basename(image_path)}"
            img_object_detection.save(save_path)  # Now you can use .save()
            print(f"  Saved annotated image to: {save_path}")
            if detections:
                img_defections.save(save_path_d)  # Now you can use .save()
                print(f"  Saved defected image to: {save_path}")
            else:
                print("  No defects detected")
            return result, detections
        else:
            print("  No objects detected in this image.")

In [36]:
test_img = '/Users/azizbek/Downloads/tok-stoyka.png'
outputs  = run_full_pipeline(img = None, image_path = test_img, save_dir="debug_crops")
print(outputs)


0: 640x640 1 yoke, 2 yoke suspensions, 1 stockbridge damper, 1 polymer insulator, 1 polymer insulator lower shackle, 307.8ms
Speed: 2.8ms preprocess, 307.8ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)
  Type of result.boxes: <class 'ultralytics.engine.results.Boxes'>
  Value of result.boxes: ultralytics.engine.results.Boxes object with attributes:

cls: tensor([ 6.,  1.,  1., 10.,  0.,  3.])
conf: tensor([0.9585, 0.9519, 0.9504, 0.9173, 0.9640, 0.7293])
data: tensor([[5.5177e+02, 2.4817e-01, 1.0419e+03, 2.5341e+02, 9.5853e-01, 6.0000e+00],
        [1.5869e+03, 3.0617e+02, 1.7950e+03, 7.5625e+02, 9.5195e-01, 1.0000e+00],
        [8.1519e+01, 2.3358e+02, 2.8312e+02, 6.6412e+02, 9.5043e-01, 1.0000e+00],
        [6.8174e+02, 2.4752e+02, 9.1425e+02, 5.4791e+02, 9.1726e-01, 1.0000e+01],
        [8.3834e+01, 2.3688e+02, 1.7490e+03, 1.0809e+03, 9.6403e-01, 0.0000e+00],
        [3.8577e+02, 9.5246e+02, 8.9732e+02, 1.0799e+03, 7.2926e-01, 3.0000e+00]])
id: None
is_track: F

ValueError: conversion from RGB to BRG not supported

In [8]:
import os, random, cv2, torch
from pathlib import Path

# ─── CONFIG ───────────────────────────────────────────────────────────────────
DEFECT_THR  = 0.5            # show box only if defect‑prob ≥ this
PAD         = 4              # pixels of padding when cropping

# simple RGB‑to‑tensor preproc matching your classifier
def to_tensor(img_bgr):
    return ( torch.from_numpy(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
             .permute(2,0,1).float()/255. ).unsqueeze(0)

@torch.inference_mode()
def detect_and_filter(image_path: str|Path):
    im_bgr   = cv2.imread(str(image_path))
    h, w     = im_bgr.shape[:2]

    # 1 ▸ DETECT PARTS
    results  = detector.predict(im_bgr, conf=0.65, device=DEVICE, verbose=False)[0]
    if len(results.boxes) == 0:                      # → nothing at all
        return im_bgr, []

    boxes_xyxy = results.boxes.xyxy.cpu().numpy()    # [N,4]
    part_names = [results.names[int(c)] for c in results.boxes.cls]

    keep, info = [], []                              # filtered indices & meta
    # 2 ▸ CROP ▸ CLASSIFY EACH BOX
    for idx, (x1,y1,x2,y2) in enumerate(boxes_xyxy.astype(int)):
        # crop with small padding
        x1p,y1p = max(x1-PAD,0), max(y1-PAD,0)
        x2p,y2p = min(x2+PAD,w-1), min(y2+PAD,h-1)
        crop    = im_bgr[y1p:y2p, x1p:x2p]

        prob = torch.sigmoid(classifier(to_tensor(crop).to(DEVICE))).item()
        if prob >= DEFECT_THR:                       # defective → keep
            keep.append(idx)
            info.append((x1,y1,x2,y2, part_names[idx], prob))

    return im_bgr, info                              # original image + kept boxes

# ─── TEST LOOP ────────────────────────────────────────────────────────────────


In [10]:
num_test_images = 5
# Path to your directory of random images
image_dir = "../data/InsPLAD-det/val/images"
image_dir = "../data/InsPLAD-fault/unsupervised_anomaly_detection/glass-insulator/test/missingcap"

# Get a list of all image files
image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

for _ in range(num_test_images):
    img_path = random.choice(image_files)
    print(f"\n--- Processing image: {os.path.basename(img_path)} ---")

    img_bgr, bad_parts = detect_and_filter(img_path)
    if not bad_parts:
        print("  No defects found.")
        continue

    # draw only “bad” boxes
    for (x1,y1,x2,y2, label, prob) in bad_parts:
        cv2.rectangle(img_bgr, (x1,y1), (x2,y2), (0,0,255), 2)          # red box
        cv2.putText(img_bgr,
                    f"{label}: {prob:.2f}",
                    (x1, max(y1-5,12)),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (0,0,255), 1, cv2.LINE_AA)
        print(f"    DEFECT {label}  p={prob:.2f}  box=({x1},{y1})–({x2},{y2})")

    save_path = f"../results/defect_detection/defects_{os.path.basename(img_path)}"
    cv2.imwrite(save_path, img_bgr)
    print(f"  Saved annotated defects‑only image to: {save_path}")


--- Processing image: Fotos 21-10-2020_DJI_0557_cadeia_isolador_vidro_1630.jpg ---
  No defects found.

--- Processing image: Fotos 21-10-2020_DJI_0537_cadeia_isolador_vidro_1615.jpg ---
  No defects found.

--- Processing image: Fotos 07-12-2020_DJI_0134_cadeia_isolador_vidro_1338.jpg ---
  No defects found.

--- Processing image: Fotos 03-12-2020_DJI_0361_cadeia_isolador_vidro_911.jpg ---
  No defects found.

--- Processing image: Fotos 03-12-2020_DJI_0361_cadeia_isolador_vidro_912.jpg ---
  No defects found.
